In [1]:
# Install specific libraries
! pip install transformers
! pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.9/485.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.7/257.7 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.9/21.9 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.0 MB

In [2]:
import numpy as np
import pandas as pd
import pycaret
import transformers
from transformers import AutoModel, BertTokenizerFast
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
# specify GPU
device = torch.device("cuda")

In [3]:
# Mount Google Drive - applicable, if working on Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/NSS_HACKATHON


/content/drive/MyDrive/NSS_HACKATHON


In [85]:
import pandas as pd

train_path = "./Task 2/train.csv"
dev_path="./Task 2/dev.csv"
test_path="./Task 2/test.csv"
# Import the CSV file
train_df = pd.read_csv(train_path)
dev_df = pd.read_csv(dev_path)
test = pd.read_csv(test_path)


In [86]:
data = pd.concat([train_df, dev_df], ignore_index=True)

# Iterate through all rows in the 'tokens' column
for index, row in data.iterrows():
    converted_list = eval(row['tokens'])
    data.at[index, 'tokens'] = converted_list

'RT'

In [87]:

max_length = max(len(row) for row in data['tokens'])

print(max_length)


86


In [89]:
data['span_start_index'] = data['span_start_index'].apply(lambda x: '[0]' if x == '[]' else x)
data['span_end_index'] = data['span_end_index'].apply(lambda x: '[0]' if x == '[]' else x)


In [90]:
import pandas as pd

# Define the function to generate the list
def generate_list(span_start_index_str, span_end_index_str, total_length=86):
    # Initialize the list with zeros
    result = [0] * total_length

    # Check if span_start_index or span_end_index is empty, if so, return the list of zeros


    # Convert string inputs to list of integers
    span_start_index = list(map(int, span_start_index_str.strip('[]').split(',')))
    span_end_index = list(map(int, span_end_index_str.strip('[]').split(',')))

    # Mark the specified ranges as 1
    for start, end in zip(span_start_index, span_end_index):
        # Include all indices in the range
        for i in range(start, end + 1):
            result[i - 1] = 1

    return result

# Create a sample DataFrame


# Iterate through each row in the DataFrame and apply the generate_list function
generated_lists = []
for index, row in data.iterrows():
    list_result = generate_list(row['span_start_index'], row['span_end_index'])
    generated_lists.append(list_result)

# Add the generated lists to the DataFrame
data['generated_list'] = generated_lists

# Check the updated DataFrame


                                                 tokens span_start_index  \
0     ["who,  may,  (or,  may,  not, ) have,  it, " ...             [43]   
1     [RT,  @Coach_Brod, : If,  you,  have,  ever,  ...              [2]   
2     [#Pharmacists,  warn,  against,  #malarial,  d...              [0]   
3     [You,  got,  to,  boil,  your,  Clorox,  to,  ...          [0, 22]   
4     [There,  is,  no,  virus, . \nAnd,  if,  there...              [0]   
...                                                 ...              ...   
6795  [it,  is,  crazy,  that,  Kony,  2012,  is,  t...              [4]   
6796  [patients,  with,  covid, 19,  have,  reported...              [0]   
6797  [Marilyn,  Monroe,  had,  an,  IQ,  of,  168, ...              [0]   
6798  [RT,  @ImBradySmith, : If,  you,  ever,  drank...              [2]   
6799  [Is,  #California's,  ability,  to,  flatten, ...             [22]   

     span_end_index                                     generated_list  
0             

In [91]:
data

,tokens,span_start_index,span_end_index,generated_list
0,"[""who, may, (or, may, not, ) have, it, "" ...",[43],[53],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[RT, @Coach_Brod, : If, you, have, ever, ...",[2],[17],"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"[#Pharmacists, warn, against, #malarial, d...",[0],[4],"[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[You, got, to, boil, your, Clorox, to, ...","[0, 22]","[20, 33]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"[There, is, no, virus, . \nAnd, if, there...",[0],[3],"[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
6795,"[it, is, crazy, that, Kony, 2012, is, t...",[4],[10],"[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
6796,"[patients, with, covid, 19, have, reported...",[0],[12],"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ..."
6797,"[Marilyn, Monroe, had, an, IQ, of, 168, ...",[0],[6],"[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6798,"[RT, @ImBradySmith, : If, you, ever, drank...",[2],[18],"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [92]:
data['tokens']

0       ["who,  may,  (or,  may,  not, ) have,  it, " ...
1       [RT,  @Coach_Brod, : If,  you,  have,  ever,  ...
2       [#Pharmacists,  warn,  against,  #malarial,  d...
3       [You,  got,  to,  boil,  your,  Clorox,  to,  ...
4       [There,  is,  no,  virus, . \nAnd,  if,  there...
                              ...                        
6795    [it,  is,  crazy,  that,  Kony,  2012,  is,  t...
6796    [patients,  with,  covid, 19,  have,  reported...
6797    [Marilyn,  Monroe,  had,  an,  IQ,  of,  168, ...
6798    [RT,  @ImBradySmith, : If,  you,  ever,  drank...
6799    [Is,  #California's,  ability,  to,  flatten, ...
Name: tokens, Length: 6800, dtype: object

In [99]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, TimeDistributed

# Convert tokens to numerical sequences
word_to_index = {word: index + 1 for index, word in enumerate(set(word for sentence in data['tokens'] for word in sentence))}
sequences = [[word_to_index.get(word, 0) for word in row] for row in data['tokens']]

# Padding sequences
max_sequence_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')

# Convert generated lists to numpy array
output_data = np.array(data['generated_list'].tolist())

# Split data into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, output_data, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define model architecture
model = Sequential([
    Embedding(input_dim=len(word_to_index) + 1, output_dim=32, input_length=max_sequence_length),
    LSTM(units=64, return_sequences=True),
    TimeDistributed(Dense(units=1, activation='sigmoid'))
])

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


Epoch 1/10
136/136 [==============================] - 14s 69ms/step - loss: 0.2976 - accuracy: 0.8497 - val_loss: 0.2369 - val_accuracy: 0.8710
Epoch 2/10
136/136 [==============================] - 8s 59ms/step - loss: 0.2089 - accuracy: 0.8962 - val_loss: 0.2195 - val_accuracy: 0.8850
Epoch 3/10
136/136 [==============================] - 7s 49ms/step - loss: 0.1718 - accuracy: 0.9212 - val_loss: 0.2346 - val_accuracy: 0.8869
Epoch 4/10
136/136 [==============================] - 8s 59ms/step - loss: 0.1406 - accuracy: 0.9395 - val_loss: 0.2418 - val_accuracy: 0.8861
Epoch 5/10
136/136 [==============================] - 7s 49ms/step - loss: 0.1160 - accuracy: 0.9528 - val_loss: 0.2698 - val_accuracy: 0.8878
Epoch 6/10
136/136 [==============================] - 8s 59ms/step - loss: 0.0972 - accuracy: 0.9617 - val_loss: 0.2766 - val_accuracy: 0.8849
Epoch 7/10
136/136 [==============================] - 8s 55ms/step - loss: 0.0831 - accuracy: 0.9679 - val_loss: 0.3049 - val_accuracy: 0.885

In [100]:
# Evaluate the model on training data
train_loss, train_accuracy = model.evaluate(X_train, y_train)
print(f'Training Loss: {train_loss}, Training Accuracy: {train_accuracy}')

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')


136/136 [==============================] - 3s 22ms/step - loss: 0.0559 - accuracy: 0.9795
Training Loss: 0.05593371391296387, Training Accuracy: 0.9794745445251465
43/43 [==============================] - 1s 21ms/step - loss: 0.3610 - accuracy: 0.8818
Test Loss: 0.361017644405365, Test Accuracy: 0.8817543387413025


In [102]:
for index, row in test.iterrows():
    converted_list = eval(row['tokens'])
    test.at[index, 'tokens'] = converted_list

In [106]:
test

,tokens
0,"[And, it, should, be, noted, that, it, ..."
1,"[China, has, nearly, won, WWIII, by, usi..."
2,"[the, democrats, want, a, market, crash, ..."
3,"[Carry, aerosol, lysol, \nAnyone, coughing,..."
4,"[According, with, Dr, Luis, Aguirre, this..."
...,...
750,"[No, evidence, ' recovered, coronavirus, pa..."
751,"[Tea, can, help, to, cure, #coronavirus]"
752,"[@oaq1212, @SamHarrisOrg, He, wants, to, ..."
753,"[we, are, going, to, look, back, and, f..."


In [103]:

max_length = max(len(row) for row in test['tokens'])

print(max_length)


60


In [123]:
test_sequences = [[word_to_index.get(word, 0) for word in row] for row in test['tokens']]

# Padding sequences
padded_test_sequences = pad_sequences(test_sequences, maxlen=86, padding='post')

# Make predictions using the trained model
predictions = model.predict(padded_test_sequences)
predictions = pd.DataFrame(predictions.squeeze(), columns=[f'position_{i+1}' for i in range(predictions.shape[1])]).values
# Save the predictions to a CSV file


24/24 [==============================] - 1s 27ms/step


In [119]:
# predictions_df = pd.DataFrame(predictions.squeeze(), columns=[f'position_{i+1}' for i in range(predictions.shape[1])])

In [126]:
rounded_predictions = np.round(predictions).astype(int)


In [127]:
rounded_predictions

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [128]:
predictions_df = pd.DataFrame(rounded_predictions, columns=[f'position_{i+1}' for i in range(rounded_predictions.shape[1])])
predictions_df.to_csv('task_2_predictions.csv', index=False)

print("Predictions saved to task_2_predictions.csv")

Predictions saved to task_2_predictions.csv


In [160]:
new_path="task_2_predictions.csv"
# Import the CSV file
sub = pd.read_csv(new_path)

In [159]:
sub

{'position_1': [1],
 'position_2': [1],
 'position_3': [0],
 'position_4': [1],
 'position_5': [1],
 'position_6': [1],
 'position_7': [1],
 'position_8': [1],
 'position_9': [1],
 'position_10': [1]}

In [165]:
import pandas as pd

# Assuming df is your DataFrame containing the positions
df = pd.DataFrame(sub)

# Convert the DataFrame into the desired format
news_df = df.values.tolist()

print(news_df)


[[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [175]:
import pandas as pd

# Assuming you have a list of lists named news_df where each sublist contains predicted values
# For example:
# news_df = [[prediction1a, prediction1b, ...], [prediction2a, prediction2b, ...], ...]

# Create a DataFrame with predictions column
predictions_df = pd.DataFrame({'predictions': news_df})

# Display the new DataFrame
print(predictions_df)


                                           predictions
0    [1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
1    [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
3    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4    [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
..                                                 ...
750  [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...
751  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
752  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
753  [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
754  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...

[755 rows x 1 columns]


In [171]:
news_df[0]

[1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [168]:
news_df.shape

AttributeError: 'list' object has no attribute 'shape'

In [153]:
new_df

,0,1
0,1,0
1,1,1
2,1,1
3,1,1
4,1,1
5,0,1
6,1,1
7,1,0
8,1,1
9,1,1


In [130]:
sub

,position_1,position_2,position_3,position_4,position_5,position_6,position_7,position_8,position_9,position_10,...,position_77,position_78,position_79,position_80,position_81,position_82,position_83,position_84,position_85,position_86
0,1,1,0,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750,1,1,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
751,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
752,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
753,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [132]:
import pandas as pd

# Define the function to convert the generated list to span indices
def list_to_span_indices(generated_list):
    span_start_index = []
    span_end_index = []
    start_index = None

    for i, val in enumerate(generated_list):
        if val == 1 and start_index is None:
            start_index = i + 1
        elif val == 0 and start_index is not None:
            span_start_index.append(start_index)
            span_end_index.append(i)
            start_index = None

    # Handle case when sequence ends with 1
    if start_index is not None:
        span_start_index.append(start_index)
        span_end_index.append(len(generated_list) - 1)

    return span_start_index, span_end_index

# Example list
generated_list = [1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0]

# Convert the list to span indices
span_start_index, span_end_index = list_to_span_indices(generated_list)

# Print the results
print("Span Start Index:", span_start_index)
print("Span End Index:", span_end_index)


,position_1,position_2,position_3,position_4,position_5,position_6,position_7,position_8,position_9,position_10,...,position_79,position_80,position_81,position_82,position_83,position_84,position_85,position_86,span_start_index,span_end_index
0,1,1,0,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,1.0,2.0
1,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.0,4.0
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,1.0,24.0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
4,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2.0,3.0


In [167]:
first_column = [row[0] for row in news_df]
print(first_column)


[1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 

In [176]:
import pandas as pd

# Define the function to convert the generated list to span indices
def list_to_span_indices(generated_list):
    span_start_index = []
    span_end_index = []
    start_index = None

    for i, val in enumerate(generated_list):
        if val == 1 and start_index is None:
            start_index = i + 1
        elif val == 0 and start_index is not None:
            span_start_index.append(start_index)
            span_end_index.append(i)
            start_index = None

    # Handle case when sequence ends with 1
    if start_index is not None:
        span_start_index.append(start_index)
        span_end_index.append(len(generated_list) - 1)

    return span_start_index, span_end_index

# Example DataFrame


# Create DataFrame

# Apply the function to the DataFrame column containing lists
predictions_df[['span_start_index', 'span_end_index']] = predictions_df['predictions'].apply(lambda x: pd.Series(list_to_span_indices(x)))

# Print the results


[[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [178]:
predictions_df.to_csv('predictions.csv', index=False)


In [179]:
predictions_df

,predictions,span_start_index,span_end_index
0,"[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 4]","[2, 20]"
1,"[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[1],[4]
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[1],[24]
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[],[]
4,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[2],[3]
...,...,...,...
750,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...",[1],[8]
751,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[],[]
752,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[],[]
753,"[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 19]","[4, 40]"


   start_index end_index
0          [0]       [0]
1       [0, 1]    [1, 1]
2       [0, 1]    [1, 1]
3       [0, 1]    [1, 1]
4       [0, 1]    [1, 1]
5          [1]       [1]
6       [0, 1]    [1, 1]
7          [0]       [0]
8       [0, 1]    [1, 1]
9       [0, 1]    [1, 1]
10      [0, 1]    [1, 1]


In [146]:
start_end_df

,start_index,end_index
0,"[1, 4, 87]","[0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
1,"[1, 87]","[0, 0, 0, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,"[1, 87]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,[2],"[0, 0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
...,...,...
750,"[1, 87]","[0, 0, 0, 0, 0, 0, 0, 8, 9, 10, 11, 12, 13, 14..."
751,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
752,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
753,"[1, 19, 87]","[0, 0, 0, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [148]:
import pandas as pd

# Define the function to convert the generated list to span indices
def list_to_span_indices(generated_list):
    span_start_index = []
    span_end_index = []
    start_index = None

    for i, val in enumerate(generated_list):
        if val == 1 and start_index is None:
            start_index = i + 1
        elif val == 0 and start_index is not None:
            span_start_index.append(start_index)
            span_end_index.append(i)
            start_index = None

    # Handle case when sequence ends with 1
    if start_index is not None:
        span_start_index.append(start_index)
        span_end_index.append(len(generated_list) - 1)

    return span_start_index, span_end_index

# Example list
generated_list = [1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0]

# Convert the list to span indices
span_start_index, span_end_index = list_to_span_indices(generated_list)

# Print the results
print("Span Start Index:", span_start_index)
print("Span End Index:", span_end_index)


Span Start Index: [1, 6, 10]
Span End Index: [3, 8, 11]


In [68]:
# import torch
# import pandas as pd
# from transformers import BertTokenizer, BertForQuestionAnswering
# from torch.utils.data import TensorDataset, DataLoader
# from sklearn.model_selection import train_test_split

# # Load the combined DataFrame containing train and dev data
# data = pd.concat([train_df, dev_df], ignore_index=True)

# # Train-Validation-Test set split into 70:15:15 ratio
# # Train-Temp split
# train_data, temp_data = train_test_split(data, random_state=2018, test_size=0.3)
# # Validation-Test split
# val_data, test_data = train_test_split(temp_data, random_state=2018, test_size=0.5)

# # Tokenize train text
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# train_encodings = tokenizer(list(train_data['tokens']), padding=True, truncation=True, return_tensors='pt')

# # Replace empty lists with [0]
# train_data['span_start_index'] = train_data['span_start_index'].apply(lambda x: '[0]' if x == '[]' else x)
# train_data['span_end_index'] = train_data['span_end_index'].apply(lambda x: '[0]' if x == '[]' else x)

# # Define a function to parse and handle empty lists
# def parse_indices(indices_str):
#     if indices_str:
#         return [int(i.strip('[]')) for i in indices_str.split(',')]
#     else:
#         return [0]

# # Convert span start and end indices to token-level labels for train data
# train_start_positions = [torch.tensor(parse_indices(indices)).view(-1) for indices in train_data['span_start_index']]
# train_end_positions = [torch.tensor(parse_indices(indices)).view(-1) for indices in train_data['span_end_index']]

# # Padding and stacking the tensors
# max_len = max(len(indices) for indices in train_start_positions + train_end_positions)
# train_start_positions = torch.stack([torch.nn.functional.pad(torch.tensor(indices), (0, max_len - len(indices))) for indices in train_start_positions])
# train_end_positions = torch.stack([torch.nn.functional.pad(torch.tensor(indices), (0, max_len - len(indices))) for indices in train_end_positions])

# # Concatenate start and end position tensors
# train_labels = torch.cat([train_start_positions, train_end_positions], dim=1)

# # Reshape the concatenated tensor into a matrix with 2 rows
# train_labels = train_labels.view(-1, 2)

# # Convert tokenized text and labels to PyTorch tensors
# train_inputs = train_encodings['input_ids']
# train_token_type_ids = train_encodings['token_type_ids']
# train_attention_mask = train_encodings['attention_mask']

# # Create TensorDataset and DataLoader for training data
# train_dataset = TensorDataset(train_inputs, train_token_type_ids, train_attention_mask, train_labels)
# train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)


AssertionError: Size mismatch between tensors

In [61]:
# # Replace empty lists with [0]
# train_data['span_start_index'] = train_data['span_start_index'].apply(lambda x: '[0]' if x == '[]' else x)
# train_data['span_end_index'] = train_data['span_end_index'].apply(lambda x: '[0]' if x == '[]' else x)


In [62]:
# train_data.isna().sum()

tokens              0
span_start_index    0
span_end_index      0
dtype: int64

In [63]:
# def parse_indices(indices_str):
#     if indices_str:
#         return [int(i.strip('[]')) for i in indices_str.split(',')]
#     else:
#         return [0]
# # Remove rows with NaN values in the 'span_start_index' and 'span_end_index' columns

# # Convert span start and end indices to token-level labels for train data
# train_start_positions = [torch.tensor(parse_indices(indices)).view(-1) for indices in train_data['span_start_index']]
# train_end_positions = [torch.tensor(parse_indices(indices)).view(-1) for indices in train_data['span_end_index']]

In [64]:
# train_inputs = train_encodings['input_ids']
# train_token_type_ids = train_encodings['token_type_ids']
# train_attention_mask = train_encodings['attention_mask']

In [65]:
# max_len = max(len(indices) for indices in train_start_positions + train_end_positions)
# train_start_positions = torch.stack([torch.nn.functional.pad(torch.tensor(indices), (0, max_len - len(indices))) for indices in train_start_positions])
# train_end_positions = torch.stack([torch.nn.functional.pad(torch.tensor(indices), (0, max_len - len(indices))) for indices in train_end_positions])

In [66]:
# # Create TensorDataset and DataLoader for training data
# train_dataset = TensorDataset(train_inputs, train_token_type_ids, train_attention_mask, train_start_positions, train_end_positions)
# train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)


In [67]:

model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

# Training loop (fine-tuning BERT for span detection)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
for epoch in range(3):  # Adjust number of epochs as needed
    model.train()
    for batch in train_dataloader:
        input_ids, token_type_ids, attention_mask, start_positions, end_positions = batch
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask,
                        start_positions=start_positions, end_positions=end_positions)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} completed")

# Save the trained model if needed
# model.save_pretrained('path_to_save_model')

# Validation and test data processing and evaluation can be done similarly

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: 0D or 1D target tensor expected, multi-target not supported